## Config & Paths - Colab Pro Optimized

In [ ]:
# ===== CONFIGURATION & PATHS =====
# Set base path for Colab vs local
if IN_COLAB:
    BASE_PATH = '/content/drive/MyDrive/Colab Notebooks/KaggleChallenge2'
else:
    BASE_PATH = os.path.dirname(os.path.abspath('.'))

# Verify base path exists
if IN_COLAB and not os.path.exists(BASE_PATH):
    print(f'❌ BASE_PATH does not exist: {BASE_PATH}')
    print(f'\nAvailable folders in /content/drive/MyDrive/:')
    import subprocess
    result = subprocess.run(['ls', '/content/drive/MyDrive/'], capture_output=True, text=True)
    print(result.stdout)
else:
    print(f'✓ BASE_PATH verified: {BASE_PATH}')

DATASET_DIR = os.path.join(BASE_PATH, 'dataset')
TRAIN_DATA_PATH = os.path.join(DATASET_DIR, 'train_data')
TEST_DATA_PATH = os.path.join(DATASET_DIR, 'test_data')
LABELS_FILE = os.path.join(DATASET_DIR, 'train_labels.csv')

# Verify dataset paths
print(f'\nDataset paths:')
print(f'  Dataset dir: {DATASET_DIR} (exists: {os.path.exists(DATASET_DIR)})')
print(f'  Train data: {TRAIN_DATA_PATH} (exists: {os.path.exists(TRAIN_DATA_PATH)})')
print(f'  Test data: {TEST_DATA_PATH} (exists: {os.path.exists(TEST_DATA_PATH)})')
print(f'  Labels file: {LABELS_FILE} (exists: {os.path.exists(LABELS_FILE)})')

# Cache directory
if IN_COLAB:
    CACHE_DIR = os.path.join(BASE_PATH, 'cache_convnext_base')
else:
    CACHE_DIR = 'data_cache_base'
os.makedirs(CACHE_DIR, exist_ok=True)

X_TRAIN_CACHE = os.path.join(CACHE_DIR, 'X_train_384.npy')
Y_TRAIN_CACHE = os.path.join(CACHE_DIR, 'y_train.npy')
X_TEST_CACHE = os.path.join(CACHE_DIR, 'X_test_384.npy')
TEST_FILES_CACHE = os.path.join(CACHE_DIR, 'test_files.pkl')

# Models directory
MODELS_DIR = os.path.join(BASE_PATH, 'models_convnext_base')
os.makedirs(MODELS_DIR, exist_ok=True)

# Submissions directory
SUBMISSION_DIR = os.path.join(BASE_PATH, 'submissions')
os.makedirs(SUBMISSION_DIR, exist_ok=True)

# ===== TRAINING CONFIGURATION - COLAB PRO (NO CONSTRAINTS) =====
TARGET_SIZE = (384, 384)  # Higher resolution for better features
BATCH_SIZE = 64  # Large batch for stable gradients
EPOCHS = 150  # Longer training with more data
KFOLDS = 5
SEED = 42

np.random.seed(SEED)
tf.random.set_seed(SEED)

print('\n✓ Configuration loaded')
print(f'  Running in: {"Colab Pro" if IN_COLAB else "Local"}')
print(f'  Cache dir: {CACHE_DIR}')
print(f'  Models dir: {MODELS_DIR}')
print(f'  Target size: {TARGET_SIZE}')
print(f'  Batch size: {BATCH_SIZE}')
print(f'  Epochs: {EPOCHS}')
print(f'  K-Folds: {KFOLDS}')

## Utility Functions

In [ ]:
# Image loading utility
def load_image(path):
    """Load image, resize, and normalize to [0, 1]."""
    img = Image.open(path).convert('RGB').resize(TARGET_SIZE)
    return np.asarray(img, dtype=np.float32) / 255.0


# Focal Loss with class-balanced weighting
class FocalLossCB(keras.losses.Loss):
    """Focal loss with per-class alpha weighting."""
    def __init__(self, alpha_vec, gamma=2.0, **kwargs):
        super().__init__(**kwargs)
        self.alpha_vec = tf.constant(alpha_vec, dtype=tf.float32)
        self.gamma = gamma

    def call(self, y_true, y_pred):
        y_pred = tf.clip_by_value(y_pred, 1e-7, 1 - 1e-7)
        pt = tf.reduce_sum(y_true * y_pred, axis=1)
        ce = -tf.reduce_sum(y_true * tf.math.log(y_pred), axis=1)
        alpha = tf.reduce_sum(y_true * self.alpha_vec, axis=1)
        focal = alpha * tf.pow(1.0 - pt, self.gamma) * ce
        return tf.reduce_mean(focal)


# MixUp augmentation for tf.data
def make_mixup_dataset(X, y, batch_size, num_classes, alpha=0.4, shuffle=True):
    """Create dataset with MixUp augmentation. y should be integer class labels."""
    y_onehot = keras.utils.to_categorical(y, num_classes=num_classes).astype(np.float32)
    
    ds = tf.data.Dataset.from_tensor_slices((X, y_onehot))
    if shuffle:
        ds = ds.shuffle(buffer_size=len(X), reshuffle_each_iteration=True)
    ds = ds.batch(batch_size)
    
    def _mix(batch_x, batch_y):
        beta = tf.random.uniform((), 0, 1)
        lam = tf.cast(tf.maximum(beta, 1.0 - beta), tf.float32)
        idx = tf.random.shuffle(tf.range(tf.shape(batch_x)[0]))
        x2 = tf.gather(batch_x, idx)
        y2 = tf.gather(batch_y, idx)
        mixed_x = lam * batch_x + (1.0 - lam) * x2
        mixed_y = lam * batch_y + (1.0 - lam) * y2
        return mixed_x, mixed_y
    
    return ds.map(_mix, num_parallel_calls=tf.data.AUTOTUNE).prefetch(tf.data.AUTOTUNE)

print('✓ Utility functions defined')

## Load & Cache Data

In [ ]:
# Load labels and fit encoder
labels_df = pd.read_csv(LABELS_FILE)
labels_df['sample_index'] = labels_df['sample_index'].astype(str)

label_encoder = LabelEncoder().fit(labels_df['label'])
num_classes = len(label_encoder.classes_)
print(f'✓ num_classes: {num_classes}')
print(f'  Classes: {label_encoder.classes_}')

# Build label mapping
def normalize_filename(s):
    s = str(s)
    if not s.startswith('img_'):
        s = 'img_' + s
    if not (s.endswith('.png') or s.endswith('.jpg') or s.endswith('.jpeg')):
        s = s + '.png'
    return s

labels_df['key'] = labels_df['sample_index'].apply(normalize_filename)
labels_map = dict(zip(labels_df['key'], labels_df['label']))

print(f'✓ Label encoder fitted with {len(labels_map)} labels')

In [ ]:
# Load or cache training data
reload_from_disk = False
if os.path.exists(X_TRAIN_CACHE) and os.path.exists(Y_TRAIN_CACHE):
    X_train = np.load(X_TRAIN_CACHE)
    y_train = np.load(Y_TRAIN_CACHE)
    if X_train.shape[1:3] != TARGET_SIZE:
        print(f'Cached X_train shape {X_train.shape[1:3]} != TARGET_SIZE {TARGET_SIZE}; reloading...')
        reload_from_disk = True
        try:
            os.remove(X_TRAIN_CACHE)
            os.remove(Y_TRAIN_CACHE)
        except Exception:
            pass
    else:
        print(f'✓ Loaded from cache: X_train={X_train.shape}, y_train={y_train.shape}')

if not os.path.exists(X_TRAIN_CACHE) or reload_from_disk:
    print('Loading X_train and y_train from disk...')
    train_files = sorted([f for f in os.listdir(TRAIN_DATA_PATH) if f.startswith('img_')])
    train_files = [f for f in train_files if f in labels_map]
    print(f'  Total train files: {len(train_files)}')

    y_train = label_encoder.transform([labels_map[f] for f in train_files])
    X_train = np.zeros((len(train_files), *TARGET_SIZE, 3), dtype=np.float32)
    for i, fname in enumerate(tqdm(train_files, desc='Loading train images')):
        X_train[i] = load_image(os.path.join(TRAIN_DATA_PATH, fname))

    np.save(X_TRAIN_CACHE, X_train)
    np.save(Y_TRAIN_CACHE, y_train)
    print(f'✓ Cached to {CACHE_DIR}')

print(f'✓ Train set: X={X_train.shape}, y={y_train.shape}')

In [ ]:
# Load or cache test data
reload_from_disk = False
if os.path.exists(X_TEST_CACHE) and os.path.exists(TEST_FILES_CACHE):
    X_test = np.load(X_TEST_CACHE)
    with open(TEST_FILES_CACHE, 'rb') as f:
        test_files = pickle.load(f)
    if X_test.shape[1:3] != TARGET_SIZE:
        print(f'Cached X_test shape {X_test.shape[1:3]} != TARGET_SIZE {TARGET_SIZE}; reloading...')
        reload_from_disk = True
        try:
            os.remove(X_TEST_CACHE)
            os.remove(TEST_FILES_CACHE)
        except Exception:
            pass
    else:
        print(f'✓ Loaded from cache: X_test={X_test.shape}, test_files={len(test_files)}')

if not os.path.exists(X_TEST_CACHE) or reload_from_disk:
    print('Loading X_test and test_files from disk...')
    test_files = sorted([f for f in os.listdir(TEST_DATA_PATH) if f.startswith('img_')])
    print(f'  Total test files: {len(test_files)}')

    X_test = np.zeros((len(test_files), *TARGET_SIZE, 3), dtype=np.float32)
    for idx, filename in enumerate(tqdm(test_files, desc='Loading test images')):
        path = os.path.join(TEST_DATA_PATH, filename)
        try:
            X_test[idx] = load_image(path)
        except Exception as e:
            print(f'Failed to load {path}: {e}')

    np.save(X_TEST_CACHE, X_test)
    with open(TEST_FILES_CACHE, 'wb') as f:
        pickle.dump(test_files, f)
    print(f'✓ Cached to {CACHE_DIR}')

print(f'✓ Test set: X={X_test.shape}, files={len(test_files)}')

## Model Architecture - ConvNeXt-Base

In [ ]:
def build_convnext_base_model(num_classes):
    """
    ConvNeXt-Base with aggressive fine-tuning for maximum accuracy.
    - Unfreezes top 80% of layers (more capacity than Tiny)
    - Deep classification head (1024→512→256)
    - Strong augmentation during training only
    """
    # Load ConvNeXt-Base base
    base = keras.applications.ConvNeXtBase(
        include_top=False,
        weights='imagenet',
        input_shape=(*TARGET_SIZE, 3),
        pooling=None
    )
    
    # Unfreeze top 80% of layers for aggressive fine-tuning
    total_layers = len(base.layers)
    freeze_until = int(total_layers * 0.20)
    
    base.trainable = True
    for i, layer in enumerate(base.layers):
        layer.trainable = (i >= freeze_until)
    
    trainable_count = sum([1 for layer in base.layers if layer.trainable])
    print(f'ConvNeXt-Base: {total_layers} layers, {trainable_count} trainable ({trainable_count/total_layers*100:.1f}%)')
    
    # Model with strong augmentation (training only)
    inputs = keras.Input(shape=(*TARGET_SIZE, 3))
    
    # Aggressive augmentation for training
    x = layers.RandomFlip('horizontal_and_vertical')(inputs, training=True)
    x = layers.RandomRotation(0.25)(x, training=True)  # ±25% rotation
    x = layers.RandomZoom(0.25)(x, training=True)  # ±25% zoom
    x = layers.RandomTranslation(0.2, 0.2)(x, training=True)  # ±20% translation
    x = layers.RandomContrast(0.3)(x, training=True)  # Contrast jitter
    
    # ConvNeXt preprocessing
    x = keras.applications.convnext.preprocess_input(x)
    
    # Backbone
    x = base(x, training=True)
    
    # Deep classification head
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.BatchNormalization()(x)
    x = layers.Dense(1024, activation='relu', kernel_regularizer=keras.regularizers.l2(1e-4))(x)
    x = layers.Dropout(0.5)(x)
    x = layers.Dense(512, activation='relu', kernel_regularizer=keras.regularizers.l2(1e-4))(x)
    x = layers.Dropout(0.4)(x)
    x = layers.Dense(256, activation='relu', kernel_regularizer=keras.regularizers.l2(1e-4))(x)
    x = layers.Dropout(0.3)(x)
    outputs = layers.Dense(num_classes, activation='softmax', dtype='float32')(x)
    
    return keras.Model(inputs, outputs, name='ConvNeXt_Base_FT')


# Build and test model
model_test = build_convnext_base_model(num_classes)
print(f'✓ ConvNeXt-Base model built')
print(f'  Total params: {model_test.count_params():,}')
trainable_params = sum([tf.size(w).numpy() for w in model_test.trainable_weights])
print(f'  Trainable params: {trainable_params:,} ({trainable_params/model_test.count_params()*100:.1f}%)')

## K-Fold Training - ConvNeXt-Base

In [ ]:
skf = StratifiedKFold(n_splits=KFOLDS, shuffle=True, random_state=SEED)
weight_paths = []

print('=' * 60)
print(f'Starting {KFOLDS}-fold cross-validation with ConvNeXt-Base')
print('=' * 60)

for fold_num, (train_idx, val_idx) in enumerate(skf.split(X_train, y_train), 1):
    print(f'\n📊 FOLD {fold_num}/{KFOLDS}')
    print('-' * 40)
    
    X_tr, X_val = X_train[train_idx], X_train[val_idx]
    y_tr, y_val = y_train[train_idx], y_train[val_idx]
    
    print(f'Train samples: {len(X_tr)}, Val samples: {len(X_val)}')
    
    # Class-balanced focal loss weights (effective number)
    class_counts = np.bincount(y_tr, minlength=num_classes)
    print(f'Class distribution: {class_counts}')
    
    beta = 0.9999
    effective_num = 1.0 - np.power(beta, class_counts)
    weights_cb = (1.0 - beta) / (effective_num + 1e-7)
    weights_cb = weights_cb / weights_cb.sum() * num_classes
    print(f'Class weights: {weights_cb.round(3)}')
    
    # MixUp training dataset
    train_ds = make_mixup_dataset(X_tr, y_tr, batch_size=BATCH_SIZE, num_classes=num_classes, alpha=0.5, shuffle=True)
    val_ds = tf.data.Dataset.from_tensor_slices((X_val, keras.utils.to_categorical(y_val, num_classes)))
    val_ds = val_ds.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
    
    # Build model
    model = build_convnext_base_model(num_classes)
    
    # AdamW optimizer with cosine decay + warmup
    total_steps = len(X_tr) // BATCH_SIZE * EPOCHS
    warmup_steps = int(total_steps * 0.15)  # 15% warmup
    
    lr_schedule = keras.optimizers.schedules.CosineDecay(
        initial_learning_rate=5e-3,  # Lower initial LR for stability
        decay_steps=total_steps - warmup_steps,
        alpha=1e-6
    )
    
    optimizer = keras.optimizers.AdamW(
        learning_rate=lr_schedule,
        weight_decay=1e-4
    )
    
    loss_fn = FocalLossCB(alpha_vec=weights_cb, gamma=2.0)
    model.compile(optimizer=optimizer, loss=loss_fn, metrics=['accuracy'])
    
    # Callbacks
    weight_path = os.path.join(MODELS_DIR, f'fold_{fold_num}_weights.weights.h5')
    weight_paths.append(weight_path)
    
    callbacks = [
        keras.callbacks.ModelCheckpoint(
            weight_path, monitor='val_accuracy', save_best_only=True, 
            save_weights_only=True, mode='max', verbose=1
        ),
        keras.callbacks.EarlyStopping(
            monitor='val_loss', patience=15, restore_best_weights=False, verbose=1
        )
    ]
    
    # Train
    print(f'\n🚀 Training fold {fold_num}...')
    history = model.fit(
        train_ds,
        validation_data=val_ds,
        epochs=EPOCHS,
        callbacks=callbacks,
        verbose=2
    )
    
    best_acc = max(history.history['val_accuracy'])
    print(f'✓ Fold {fold_num} best val accuracy: {best_acc:.4f}')
    
    # Cleanup
    del X_tr, X_val, y_tr, y_val, train_ds, val_ds, model, history
    tf.keras.backend.clear_session()
    
print('\n' + '=' * 60)
print(f'✓ {KFOLDS}-fold training completed!')
print('=' * 60)

## Test & Submit

In [ ]:
# Ensemble predictions across folds
print("\n🔮 Inference: Ensembling fold predictions...")
all_preds = []
for fold_num, weight_path in enumerate(weight_paths, 1):
    print(f"Loading fold {fold_num} weights from: {weight_path}")
    model = build_convnext_base_model(num_classes)
    model.load_weights(weight_path)
    preds = model.predict(X_test, batch_size=BATCH_SIZE, verbose=0)
    all_preds.append(preds)
    tf.keras.backend.clear_session()

# Average predictions
ensemble_preds = np.mean(all_preds, axis=0)

# Argmax to class indices, then inverse transform to labels
pred_indices = np.argmax(ensemble_preds, axis=1)
pred_labels = label_encoder.inverse_transform(pred_indices)

# Build submission DataFrame
submission = pd.DataFrame({
    'filename': test_files,
    'label': pred_labels
})

# Save submission
out_name = f'submission_convnext_base_{KFOLDS}fold.csv'
sub_path_drive = os.path.join(SUBMISSION_DIR, out_name)
sub_path_local = os.path.join(BASE_PATH, out_name)
os.makedirs(SUBMISSION_DIR, exist_ok=True)
submission.to_csv(sub_path_drive, index=False)
submission.to_csv(sub_path_local, index=False)
print(f"\n✓ Submission saved:")
print(f"  Drive: {sub_path_drive}")
print(f"  Local: {sub_path_local}")
print(f"\nPrediction distribution:")
print(submission['label'].value_counts())